# Test des formulations incertaines

## Vérification du fonctionnement de la formulation

#### Visualisation de la convergence

In [ ]:
function visualize_convergence(graph, B, logs, N_values, K)
    """
    Fonction qui visualise la convergence de la solution en fonction du nombre de simulations N.
    
    Entrées :
        - graph    : graph   - Graphe étudié
        - B        : Integer - Budget alloué aux tests croisés
        - logs     : Boolean - Boolean déterminant si les sorties de la résolution du PLNE sont affichées
        - N_values : array   - Liste de valeurs de N à tester
        - K        : Integer - Longueur maximale des cycles (2 ou 3)
    """
    if K == 2
        # Récupération des arcs et sommets du graphe restreint aux cycles de taille 2
        C_2,V_2,C_v,A_2 = get_cycles_2(graph)
        graph[2] = A_2
    else 
        # Récupération des arcs et sommets du graphe restreint aux cycles de taille 3
        C,C_a,C_v,U,A_3,V_3 = get_cycles_3(graph)
        graph[2] = A_3
    end

    costs_obj = []

    # Simulation de maximum(N_values) scénarios, sont nous prendrons une partiede plus en plus grande au long du test
    S = simulate_scenarios(graph,maximum(N_values));

    for (i,N) in enumerate(N_values)
        # Récupération des N premiers scénarios simulés
        s = S[1:N]
        
        # Résolution du problème
        if K == 2
            obj, model,timer = uncertain_model_2(C_2,V_2,C_v, s, B, logs)
        else
            obj, model,timer = uncertain_model_3(A_3,V_3,C,C_a,C_v,U, s, B, logs)
        end
            
        push!(costs_obj,obj)

        #Affichage de l'avancement du calcul (barre d'avancement)
        progress = floor(Int, i / length(N_values) * 20)  
        barre = "■" ^ progress                  
        vide = " " ^ (20 - progress)            

        # Affiche la barre et efface la ligne précédente
        print("\r[" * barre * vide * "] "*string(i)*"/"*string(length(N_values)))  
        flush(stdout)  

    end
    
    plot(N_values,costs_obj,legend=false)
    ylabel!("z_hat")
    xlabel!("N (nombre de scénarios)")
    title!("Evolution de z_hat en fonction de N (K = "*string(K)*")",titlefont = font(10))
    
end

#### Evolution de l'espérance

In [ ]:
function evol_expectation(graph, B, M, N_values, K)
    """
    Fonction qui visualise l'évolution de la moyenne empirique de la solution en fonction du nombre de simulations N.
    
    Entrées :
        - graph    : graph   - Graphe étudié
        - B        : Integer - Budget alloué aux tests croisés
        - M        : Integer - Nombre de fois où l'on résout le problème pour estimer la moyenne empirique de la solution
        - N_values : array   - Liste de valeurs de N à tester
        - K        : Integer - Longueur maximale des cycles (2 ou 3)
    """
    if K == 2
        # Récupération des arcs et sommets du graphe restreint aux cycles de taille 2
        C_2,V_2,C_v,A_2 = get_cycles_2(graph)
        graph[2] = A_2
    else 
        # Récupération des arcs et sommets du graphe restreint aux cycles de taille 3
        C,C_a,C_v,U,A_3,V_3 = get_cycles_3(graph)
        graph[2] = A_3
    end
    
    # Liste contenant la valeur objectif moyenne pour chaque N
    E_costs = []

    # On réalise M simulations de max(N_values) scénarios
    S = []
    for i in Array(1:M)
        s = simulate_scenarios(graph,maximum(N_values));
        push!(S,s)
    end

    for (j,N) in enumerate(N_values)

        E_obj = 0
        # On résout M fois le PLNE pour estimer l'espérance de la solution
        for i in Array(1:M)
            # Récupération des N premiers scénarios de la simulation i
            s = S[i][1:N]
            
            # Résolution du problème
            if K == 2 
                obj, model,timer = uncertain_model_2(C_2,V_2,C_v, s, B,false,true)
            else
                obj, model,timer = uncertain_model_3(A_3,V_3,C,C_a,C_v,U, s, B,false)
            end
            
            # Mise à jour de la valeur objectif moyenne
            E_obj += 1/M*obj
        end

        push!(E_costs,E_obj)

        #Affichage de l'avancement du calcul (barre d'avancement)
        progress = floor(Int, j / length(N_values) * 20)  
        barre = "■" ^ progress                  
        vide = " " ^ (20 - progress)            

        # Affiche la barre et efface la ligne précédente
        print("\r[" * barre * vide * "] "*string(j)*"/"*string(length(N_values)))  
        flush(stdout)  

    end
    
    plot(N_values,E_costs,legend=false)
    ylabel!("Espérance estimée de z_hat")
    xlabel!("N (nombre de scénarios)")
    title!("Evolution de l'espérance de z_hat (B = "*string(B)*", K = "*string(K)*")",titlefont = font(10))
    
end

#### Convergence des solutions

In [ ]:
function x_convergence(graph, N_values, M, B, K)
    """
    Fonction qui visualise la convergence des solutions optimales des tests croisés à effectuer (x_hat).
    
    Entrées :
        - graph    : graph   - Graphe étudié
        - N_values : array   - Liste des nombres de scénarios à tester
        - M        : Integer - Nombre de réalisations de x_hat sur lesquelles la moyenne est réalisée
        - B        : Integer - Budget alloué aux tests croisés
        - K        : Integer - Longueur maximale des cycles (2 ou 3)
    """
    if K == 2
        # Récupération des arcs et sommets du graphe restreint aux cycles de taille 2
        C_2,V_2,C_v,A_2 = get_cycles_2(graph)
        graph[2] = A_2
    else 
        # Récupération des arcs et sommets du graphe restreint aux cycles de taille 3
        C,C_a,C_v,U,A_3,V_3 = get_cycles_3(graph)
        graph[2] = A_3
    end

    # Création de M simulations de max(N_values) scénarios
    S = []
    for i in Array(1:M)
        s = simulate_scenarios(graph,maximum(N_values));
        push!(S,s)
    end
    
    # Matrice contenant les variables x_N, pour les M réalisations
    X = [ [] for i in 1:length(N_values), j in 1:M ]
    
    # Liste contenant les distances entre x_N et x_Nmax, moyennées sur les M réalisations
    eps = []

    for (i,n) in enumerate(N_values)

        #Affichage de l'avancement du calcul (barre d'avancement)
        progress = floor(Int, i / length(N_values) * 20)  
        barre = "■" ^ progress                  
        vide = " " ^ (20 - progress)            

        # Affiche la barre et efface la ligne précédente
        print("\r[" * barre * vide * "] "*string(i)*"/"*string(length(N_values)))  
        flush(stdout)

        # Distance entre x_N et x_Nmax moyennée sur M réalisations
        e = 0
        
        for j in Array(1:M)

            # Récupération des n premiers scénarios de la simulation M
            s = S[j][1:n]
            
            # Résolution du problème
            if K == 2 
                obj, model,timer = uncertain_model_2(C_2,V_2,C_v, s, B,false)
            else
                obj, model,timer = uncertain_model_3(A_3,V_3,C,C_a,C_v,U, s, B,false)
            end

            X[i,j] = Array(value.(model[:x]))
            
            # Mise à jour de la distance moyenne entre x_N et x_Nmax
            e += 1/M * sum(abs.(X[1,j]-X[i,j]))

        end

        push!(eps, e)

    end
        
    plot(N_values,eps,legend=false)
    ylabel!("Ecart entre x_final et x")
    xlabel!("N (nombre de scénarios)")
    title!("Convergence des solutions x",titlefont = font(10))
        
end

#### Comparaison avec les formulations compactes

In [ ]:
function compare_compact()
    """
    Fonction qui compare la valeur objectif rendue sur un graphe déterministe par la formulation avec incertitudes 
    et la formlation compacte.
    """
    # Construction des graphes de test, avec une distribution "NoFailure" déterministe
    graphs = load_graphs_from_dat(["KEP_031","KEP_071","KEP_111"],"NoFailure")

    ## Comparaison pour K = 2   
    # Comptage du nombre de fois où la même solution est rendue pour les 3 graphes
    nb_egal_2=0
    for graph in graphs
        # Formulation avec incertitudes de taille 2
        B = graph[1]
        C_2,V_2,C_v,A_2 = get_cycles_2(graph)
        S = simulate_scenarios(graph,1)
        obj, model,timer = uncertain_model_2(C_2,V_2,C_v, S, B, false)
        
        # Formulation compacte de taille 2
        graph[3] = S[1] # Mise à jour de la matrice des poids W
        cost2, model2 = compact_cycle_2(graph,false)
    
        nb_egal_2 += 1*(obj==cost2)
    end

    ## Comparaison pour K = 3
    # Comptage du nombre de fois où la même solution est rendue pour les 3 graphes
    nb_egal_3 = 0
    # On ne teste pas sur l'instance KEP_111, car trop longue à résoudre
    for graph in graphs[1:2]
    # Formulation avec incertitudes de taille 3
        B = graph[1]
        C,C_a,C_v,U,A_3,V_3 = get_cycles_3(graph)
        S = simulate_scenarios(graph,1)
        obj, model,timer = uncertain_model_3(A_3,V_3,C,C_a,C_v,U, S, B, false)
    
        # Formulation compacte de taille 3
        graph[3]=S[1] # mise à jour de la matrice des poids W
        cost3, model3 = compact_cycle_K(graph,3,false)

        nb_egal_3 += 1*(obj==cost3)
    end
    
    println("Proportion de valeurs objectif égales pour K = 2 : ",nb_egal_2/4*100," %")
    println("Proportion de valeurs objectif égales pour K = 3 : ",nb_egal_3/3*100," %")
end

#### Comparaison K = 2 vs K = 3

In [ ]:
function compare_2_3(N, liste_graphs)
    """
    Fonction qui compare les valeurs objectif (du problème initial et du recours) pour K = 2 et K = 3.
    
    Entrées :
        - N            : Integer - Nombre de scénarios simulés
        - liste_graphs : array   - Liste des graphes testés
    """
    # Construction des graphes
    graphs = load_graphs_from_dat(liste_graphs)

    # Comptage du nombre de fois ou z_hat_2 <= z_hat_3
    test_hat = 0
    # Comptage du nombre de fois ou z_barre_2 <= z_barre_3
    test_barre = 0
    
    for (i,graph) in enumerate(graphs)

        # Simulations des scénarios pour la résolution du problème initial et du recours
        S = simulate_scenarios(graph,N)
        S_prime = simulate_scenarios(graph,500);
        B = graph[1] # on fixe un budget égal au nombre de sommets dans le graphe

        # Formulation avec incertitudes de taille 2
        C_2,V_2,C_v,A_2 = get_cycles_2(graph)
        obj2, model,timer = uncertain_model_2(C_2,V_2,C_v, S, B, false)
        Obj2, model, timer = pb_recours_2(C_2,V_2,C_v, S_prime, false, value.(model[:x]))

        # Formulation avec incertitudes de taille 3
        C,C_a,C_v,U,A_3,V_3 = get_cycles_3(graph)
        obj3, model,timer = uncertain_model_3(A_3,V_3,C,C_a,C_v,U, S, B,false, false, 600)
        Obj3, model, timer = pb_recours_3(A_3,V_3,C,C_a,C_v,U, S_prime, false, value.(model[:x]))
        
        # Incrémentation des compteurs
        test_hat += 1*(obj2 <= obj3)
        test_barre += 1*(Obj2 <= Obj3)
        
        println("Graphe ", liste_graphs[i]," :")
        println("- z_hat(K=2) : ",round(Obj2,digits=3),"               - z_barre(K=2) : ",round(obj2,digits=3))
        println("- z_hat(K=3) : ",round(Obj3,digits=3),"              - z_barre(K=3) : ",round(obj3,digits=3))
    end
    println("Proportion de cas où l'inégalité est respectée : ")
    println("   - Pour z_hat   : ", test_hat/length(liste_graphs)*100, " %")
    println("   - Pour z_barre : ", test_barre/length(liste_graphs)*100, " %")
end

## Qualités de l'approche utilisée

#### Comparaison avec problème aux valeurs moyennes

In [ ]:
function compare_EV(graphs, graph_names, B, N_values, K)
    """
    Fonction qui compare la solution trouvée avec la formulation par recours et celle aux valeurs moyennes.
    
    Entrées :
        - graphs      : array   - Liste des graphes testés
        - graph_names : array   - Noms des graphes testés
        - B           : Integer - Budget alloué aux tests croisés
        - N_values    : array   - Liste de la valeur de N utilisée pour chaque graphe
        - K           : Integer - Longueur maximale des cycles (2 ou 3)
    """
    # Stockage des valeurs objectif rendues par les deux formlations pour chaque graphe
    Z_uncertain = []
    Z_EV = []
    # Stockage du test (z_hat >= z_EV) pour chaque graphe
    tests = []

    for (i,graph) in enumerate(graphs)
        
        n = graph[1]
        # Matrice contenant l'espérance des scénarios, i.e. 1 - W (matrice 1 - matrice de poids du graphe)
        E_xi = ones(n,n) - graph[3]
        
        N = N_values[i]
        
        if K == 2
            # Récupération des arcs et sommets du graphe restreint aux cycles de taille 2
            C_2,V_2,C_v,A_2 = get_cycles_2(graph)
            graph[2] = A_2
        else 
            # Récupération des arcs et sommets du graphe restreint aux cycles de taille 3
            C,C_a,C_v,U,A_3,V_3 = get_cycles_3(graph)
            graph[2] = A_3
        end
       
        # Simulation de N scénarios pour résoudre le problème
        S = simulate_scenarios(graph,N);
        # Simulation de 10*N scénarios pour résoudre le prblèmme de recours
        S_prime = simulate_scenarios(graph,10*N);
            
        # Résolution du problème puis du recours, pour la formulation avec incertitudes et EV
        if K == 2
            obj, model,timer = uncertain_model_2(C_2,V_2,C_v, S, B[i],false,true)
            z_uncertain, model_2, timer = pb_recours_2(C_2,V_2,C_v, S_prime, false, value.(model[:x]))
                
            obj, model_EV, timer = uncertain_model_2(C_2,V_2,C_v, [E_xi], B[i], false, true)
            z_EV, model_EV_2, timer = pb_recours_2(C_2,V_2,C_v, S_prime, false, value.(model_EV[:x]))
        else 
            obj, model,timer = uncertain_model_3(A_3,V_3,C,C_a,C_v,U, S, B[i],false,false,2000)
            z_uncertain, model_2, timer = pb_recours_3(A_3,V_3,C,C_a,C_v,U, S_prime, false, value.(model[:x]))
                
            obj, model_EV, timer = uncertain_model_3(A_3,V_3,C,C_a,C_v,U, [E_xi], B[i], false, true)
            z_EV, model_EV_2, timer = pb_recours_3(A_3,V_3,C,C_a,C_v,U, S_prime, false, value.(model_EV[:x]),true)
        end
    

        push!(Z_uncertain, z_uncertain)
        push!(Z_EV, z_EV)
        push!(tests, z_uncertain >= z_EV)

        println("*** Graphe ",graph_names[i]," ***")
        println(" - Valeur optimale approchée (sous incertitudes) : ", round(z_uncertain,digits=3))
        println(" - Valeur optimale approchée (expectation value) : ", round(z_EV,digits=3))
        println(" - z* >= Z_EV ? ", z_uncertain >= z_EV)
        println("")
        
    end
    
    println("L'inégalité est vraie dans ", 100* sum(tests)/length(tests),"% des cas testés.")
    
end

#### Comparaison avec approche gencol

In [ ]:
function compare_gencol(graphs, graph_names, B_values, N_values, K)
    """
    Fonction qui compare la solution trouvée avec la formulation par recours, celle aux valeurs moyennes et la génération de colonne.
    
    Entrées :
        - graphs      : array   - Liste des graphes testés
        - graph_names : array   - Noms des graphes testés
        - B_values    : array   - Liste du budget alloué aux tests croisés pour chaque graphe
        - N_values    : array   - Liste de la valeur de N utilisée pour chaque graphe
        - K           : Integer - Longueur maximale des cycles (2 ou 3)
    """
    # On parcourt les graphes souhaités
    for (i,graph) in enumerate(graphs)
        
        if K == 2
            # Récupération des arcs et sommets du graphe restreint aux cycles de taille 2
            C,V,C_v,A = get_cycles_2(graph)
        else
            # Récupération des arcs et sommets du graphe restreint aux cycles de taille 3
            C, C_a, C_v, U, A, V = get_cycles_3(graph)
        end
        
        # Mise à jour de la liste des arcs du graphe après la restriction aux cycles de taille K
        graph[2] = A
        # Matrice contenant l'espérance des scénarios, i.e. 1 - W (matrice 1 - matrice de poids du graphe)
        E_xi = ones(graph[1],graph[1]) - graph[3]

        # Construction du graphe utilisé pour résoudre la génération de colonne
        graph_GC = [graph[1],A,E_xi,graph[4],graph[5]]
        
        # Initialisation des arcs à tester
        tests = []
        # Liste des arcs disponibles
        A_actuel = Set(graph[2])
        # Nombre de tests croisés déjà choisis
        b = 0
        B = B_values[i]
        
        # Tant que le budget max n'est pas atteint
        while b < B
            # Résolution de la génération de colonne
            model_rlp, cycles, costs, iter, C_hat_v = heuristique_gencol(graph_GC,K,"2",1e-6,"BF",0);
            # Récupération des arcs optimaux
            arcs = vcat(cycles...)
            # On retire les arcs de A
            A_actuel = setdiff(A_actuel, Set(arcs))
                
            # Mise à jour du graphe graph_GC
            graph_GC[2] = collect(A_actuel)
            
            # Ajout des arcs dans la liste des tests à réaliser
            if b + length(arcs) > B
                push!(tests, arcs[1:(B-b)])
                b = B
            else
                push!(tests, arcs)
                b += length(arcs)
            end
        end
        
        # Liste contenant les tests croisés à réaliser
        tests = vcat(tests...)

        # Reconstruction de la variable x de nos modèles avec incertitudes, 
        ## en utilisant le résultat donné par la génération de colonne
        x_tests = Dict()
        if K == 2
            tests = tests[1:2:length(tests)]

            # Remplir le dictionnaire
            for idx in C
                if idx in tests
                    x_tests[idx] = 1.0
                else 
                    x_tests[idx] = 0.0
                end
            end
        else
            # Remplir le dictionnaire
            for idx in A
                if idx in tests
                    x_tests[idx] = 1.0
                else 
                    x_tests[idx] = 0.0
                end
            end
        end

        # Simulation des scénarios pour résoudre le recours
        S_prime = simulate_scenarios(graph,1000)
        
        # Résolution du recours avec les tests croisés obtenus par la génération de colonne
        if K == 2
            Obj, model, timer = pb_recours_2(C,V,C_v, S_prime, false, x_tests)
        else
            Obj, model, timer = pb_recours_3(A,V,C, C_a, C_v, U, S, false, x_tests)
        end
            
        println("\nIntance ",graph_names[i]," : ")
        println("     Méthode par génération de colonnes : ", Obj)

        # Paramètres de la résolution
        N = 100
        # Simulation des scénarios pour résoudre la formulation avec incertitudes et aux valeurs moyennes
        S = simulate_scenarios(graph,N);
        
        if K == 2
            # Résolution de la formulation avec incertitudes puis du recours
            obj, model, timer = uncertain_model_2(C,V,C_v, S, B,false)
            Obj, model, timer = pb_recours_2(C,V,C_v, S_prime, false, value.(model[:x]));

            # Résolution de la formulation aux valeurs moyennes (EV)
            obj, model_EV, timer = uncertain_model_2(C,V,C_v, [E_xi], B, false, true)
            z_EV, model_EV_2, timer = pb_recours_2(C,V,C_v, S_prime, false, value.(model_EV[:x]))
        else
            # Résolution de la formulation avec incertitudes puis du recours
            obj, model,timer = uncertain_model_3(A,V,C,C_a,C_v,U, S, B[i],false)
            z_uncertain, model_2, timer = pb_recours_3(A,V,C,C_a,C_v,U, S_prime, false, value.(model[:x]))
                
            # Résolution de la formulation aux valeurs moyennes (EV)
            obj, model_EV, timer = uncertain_model_3(A,V,C,C_a,C_v,U, [E_xi], B[i], false, true)
            z_EV, model_EV_2, timer = pb_recours_3(A,V,C,C_a,C_v,U, S_prime, false, value.(model_EV[:x]),true)
        end
        
        println("     Problème aux valeurs moyennes       : ", z_EV)
        println("     Méthode avec recours                : ", Obj)
        
    end
end

#### Visualisation de $\overline{z}$ en fonction de $B$

In [1]:
function compare_EVPI(graph, N, B_values, K)
    """
    Fonction qui visualise le nombre espéré de transplantations en fonction du budget alloué aux tests croisés.
    
    Entrées :
        - graph    : graph   - Graphe étudié
        - N        : Integer - Nombre de scénarios simulés
        - B_values : array   - Liste des valeurs de budget testées
        - K        : Integer - Longueur maximale des cycles (2 ou 3)
    """
    if K == 2
        # Récupération des arcs et sommets du graphe restreint aux cycles de taille 2
        C_2,V_2,C_v,A_2 = get_cycles_2(graph)
        graph[2] = A_2
    else 
        # Récupération des arcs et sommets du graphe restreint aux cycles de taille 3
        C,C_a,C_v,U,A_3,V_3 = get_cycles_3(graph)
        graph[2] = A_3
    end
    
    # Initialisation du vecteur des coûts
    costs = []

    # Simulation des scénarios pour le problème initial et de recours
    S = simulate_scenarios(graph,N);
    S_prime = simulate_scenarios(graph,10*N);

    for (i,B) in enumerate(B_values)
        
        # Résolution de la formulation avec incertitudes puis du recours
        if K == 2
            obj, model, timer = uncertain_model_2(C_2,V_2,C_v, S, B, false)
            Obj, model, timer = pb_recours_2(C_2,V_2,C_v, S_prime, false, value.(model[:x]))
        else
            obj, model,timer = uncertain_model_3(A_3,V_3,C,C_a,C_v,U, S, B,false)
            Obj, model, timer = pb_recours_3(A_3,V_3,C,C_a,C_v,U, S_prime, false, value.(model[:x]))
        end

        push!(costs,Obj)

        #Affichage de l'avancement du calcul (barre d'avancement)
        progress = floor(Int, i / length(B_values) * 20)  
        barre = "■" ^ progress                  
        vide = " " ^ (20 - progress)            

        # Affiche la barre et efface la ligne précédente
        print("\r[" * barre * vide * "] "*string(i)*"/"*string(length(B_values)))  
        flush(stdout)  

    end
    
    plot(B_values,costs,legend=false)
    ylabel!("Estimation du nombre espéré de transplantations")
    xlabel!("B (Budget)")
    title!("Evolution de z* (N = "*string(N)*", K = "*string(K)*")",titlefont = font(10))
    
end

compare_EVPI (generic function with 1 method)

## Analyse des choix de modélisation

#### Choix de la distribution des probabilités d'échec

In [ ]:
function compare_distrib(N,B,graph_name,distrib_1, distrib_2)
    """
    Fonction qui compare les solutions optimales pour deux distributions différentes.
    
    Entrées :
        - N          : Integer - Nombre de scénarios simulés
        - B          : Integer - Budget alloué aux tests croisés
        - graph_name : String  - Nom du graphe utilisé
        - distrib_1  : String  - Nom de la première distribution
        - distrib_2  : String  - Nom de la deuxième distribution
    """
    logs = false
    # Construction des 2 graphes avec les distributions rentrées en paramètres
    graph_1 = load_graphs_from_dat([graph_name], distrib_1)[1]
    graph_2 = load_graphs_from_dat([graph_name], distrib_2)[1]

    ## Graphe 1 (distribution n°1)
    C_2,V_2,C_v,A_2 = get_cycles_2(graph_1)
    graph_1[2] = A_2
    
    # Simulation des scénarios
    S = simulate_scenarios(graph_1,N)
    S_prime = simulate_scenarios(graph_1,10*N)
    
    # Récupération des valeurs optimales de la variable x et de la valeur objectif
    obj, model, timer_1 = uncertain_model_2(C_2,V_2,C_v, S, B, logs)
    X = value.(model[:x])
    Obj, model, timer = pb_recours_2(C_2,V_2,C_v, S_prime, logs, X)

    # Récupération des arcs solution
    arcs = []
    for (i,j) in axes(X)[1]
        if abs(X[(i,j)]-1) < 1e-6
            push!(arcs,(i,j))
        end
    end
        
    println("z_",distrib_1," = ", Obj)

    ## Graphe 2 (distribution n°2)
    C_2,V_2,C_v,A_2 = get_cycles_2(graph_2)
    graph_2[2] = A_2
        
    # Simulation des scénarios
    S = simulate_scenarios(graph_2,N)
    S_prime = simulate_scenarios(graph_2,10*N)
        
    # Récupération des valeurs optimales de la variable x et de la valeur objectif
    obj_2, model, timer_2 = uncertain_model_2(C_2,V_2,C_v, S, B, logs, false, 400)
    X = value.(model[:x])
    Obj_2, model, timer = pb_recours_2(C_2,V_2,C_v, S_prime, logs, X)

    # Récupération des arcs solution
    arcs_2 = []
    for (i,j) in axes(X)[1]
        if abs(X[(i,j)]-1) < 1e-6
            push!(arcs_2,(i,j))
        end
    end

    # Arcs choisis pour les tests croisés communs aux deux graphes
    tests_communs = intersect(Set(arcs),Set(arcs_2))     

    println("z_",distrib_2," = ", Obj_2)
    println("Proportion de tests croisés communs : ",length(tests_communs)/B*100," %")
    println("Temps de résolution")
    println("  - ",distrib_1, " : ", timer_1.time, " s")
    println("  - ",distrib_2, " : ", timer_2.time, " s")
end

#### Tentative d'amélioration de notre PLNE

In [ ]:
function compare_formulation_eff()
    """
    Fonction qui compare le temps de calcul et la valeur optimale pour le PLNE classique et PLNE modifié.
    """
    # Instances choisies
    instances = ["KEP_031","KEP_071"]

    nb_test_par_instance = 15
    
    # Paramètres de la résolution
    N_list = [200,100]
    B_list = [32,64]

    # Stockage des temps d'éxecution pour chaque instance avec la formulation classique
    time = [[] for j in Array(1:length(instances))]
    # Stockage des temps d'éxecution pour chaque instance avec la formulation modifiée
    time_eff = [[] for j in Array(1:length(instances))]

    # Stockage des valeurs objectif pour chaque instance avec la formulation classique
    sol = [[] for j in Array(1:length(instances))]
    # Stockage des valeurs objectif pour chaque instance avec la formulation modifiée
    sol_eff = [[] for j in Array(1:length(instances))]

                                    
    for k in Array(1:length(instances)) 

        instance = instances[k]
        N = N_list[k]
        B = B_list[k]

        for i in Array(1:nb_test_par_instance)
            # Création de l'instance
            graph = load_graphs_from_dat([instance])[1]

            # Récupération des arcs et sommets du graphe restreint aux cycles de taille 2
            C_2,V_2,C_v,A_2 = get_cycles_2(graph)
            graph[2] = A_2

            # Simulation des scénarios
            S = simulate_scenarios(graph,N);

            # Résolution du problème avec les deux formulations
            obj, model, timer = uncertain_model_2(C_2,V_2,C_v, S, B,false)
            obj_eff, model_eff, timer_eff = uncertain_model_2_eff(C_2,V_2,C_v, S, B,false)
            
            # Stockage des temps de résolution
            push!(time[k],timer[2])
            push!(sol[k],obj) 

            push!(time_eff[k],timer_eff[2])
            push!(sol_eff[k],obj_eff) 

            # Affichage de l'avancement du calcul (barre d'avancement)
            progress = floor(Int, (i / (2*nb_test_par_instance) + (k-1)/2)* 20)  
            barre = "■" ^ progress                  
            vide = " " ^ (20 - progress)            

            # Affiche la barre et efface la ligne précédente
            print("\r[" * barre * vide * "] "*string(k)*"/"*string(length(instances)))  
            flush(stdout) 
        end
    end
                                            
    println("\nPour l'instance ",instances[1]," : ")
    println("  - Formulation classique : ", round(mean(time[1]),digits=3),"s en moyenne")
    println("  - Formulation modifiée  : ", round(mean(time_eff[1]),digits=3),"s en moyenne")

    println("\n \nPour l'instance ",instances[2]," : ")
    println("  - Formulation classique : ", round(mean(time[2]),digits=3),"s en moyenne")
    println("  - Formulation modifiée  : ", round(mean(time_eff[2]),digits=3),"s en moyenne")

    plot1 = plot(Array(1:nb_test_par_instance), [time[1], time_eff[1]], label=["Formulation classique" "Formulation modifiée"], title=instances[1], xlabel="n° du test", ylabel="Temps (s)")
    plot2 = plot(Array(1:nb_test_par_instance), [time[2], time_eff[2]], label=["Formulation classique" "Formulation modifiée"], title=instances[2], xlabel="n° du test", ylabel="Temps (s)")
    plot3 = plot(Array(1:nb_test_par_instance), [sol[1], sol_eff[1]], label=["Formulation classique" "Formulation modifiée"], xlabel="n° du test", ylabel="Z_hat")
    plot4 = plot(Array(1:nb_test_par_instance), [sol[2], sol_eff[2]], label=["Formulation classique" "Formulation modifiée"], xlabel="n° du test", ylabel="Z_hat")                                        
    plot!(plot1, plot2, plot3, plot4, layout=(2, 2), size=(800, 500))  
                                        
end

#### Contrainte "éthique" sur les tests croisés

In [ ]:
function compare_formulation_ethic(graph_name)
    """
    Fonction qui compare le nb espéré de transplantations entre les 2 formulations du PLNE.
    
    Entrée :
        - graph_name : String - Nom du graphe utilisé
    """
    # Création du graphe
    graph = load_graphs_from_dat([graph_name])[1]
    
    # Récupération des arcs et sommets du graphe restreint aux cycles de taille 2
    C_2,V_2,C_v,A_2 = get_cycles_2(graph)
    graph[2] = A_2
      
    # Paramètres de la résolution
    N = 100
    M = 5

    # Stockage de M simulations contenant N (ou 10*N pour le recours) scénarios chacune
    S = []
    S_prime = []

    ## Pour chacune des M simulations, on souhaite trouver le budget B minimal tel qu'il existe une solution
    ## vérifiant la contrainte ajoutée

    # Valeurs de budget à tester
    B_tests = Array(graph[1]:100)
    # Stockage du budget minimal réalisable pour les M simulations
    B_mins = []
    for m in Array(1:M)
        # Simulation de scénarios
        s = simulate_scenarios(graph,N);
        s_prime = simulate_scenarios(graph,10*N);
        
        push!(S,s)
        push!(S_prime,s_prime)
        
        # Tant qu'il n'y a pas de solution au problème, on augmente le budget de 1 en 1
        #status = INFEASIBLE
        #ind = 1
        #while status == INFEASIBLE
        #    obj_ethic,model_ethic,timer_ethic = uncertain_model_2_ethic(C_2,V_2,C_v, s, B_tests[ind],false)
        #    status = termination_status(model_ethic)
        #    ind += 1
        #end
        # Récupération du B réalisable minimal pour la simulation m
        #B_min = B_tests[ind]
        #push!(B_mins,B_min)
    end
    
    # Pour la comparaison des 2 formulations, on fait varier B de B_min jusqu'au nombre d'arcs du graphe
    B_min = 42  # Budget minimal pour attribuer un test à tous les malades
    B_values = Array(max(B_min):20:length(A_2))
    push!(B_values,length(A_2))
    
    ## Formulation classique
    # Proportion de sommets dans les tests croisés pour chaque budget
    prop_sommets = []
    # Valeur objectif pour chaque budget
    Obj_classic = []
    # Valeur objectif du recours pour chaque budget
    Obj_classic_recours = []

    ## Formulation modifiée
    # Proportion de sommets dans les tests croisés pour chaque budget
    prop_sommets_ethic = []
    # Valeur objectif pour chaque budget
    Obj_ethic = []
    # Valeur objectif du recours pour chaque budget
    Obj_ethic_recours = []
    
    for b in Array(1:length(B_values))
        
        B = B_values[b]
        
        prop = 0
        prop_ethic = 0
        obj_classic = 0
        obj_ethic = 0
        obj_classic_recours = 0
        obj_ethic_recours = 0
        
        # Pour chaque budget b, on moyenne les résultats sur M simulations différentes
        for m in Array(1:M)
        
            # Résolution des formulations classique et modifiée
            val, model, timer = uncertain_model_2(C_2,V_2,C_v, S[m], B,false)
            val_ethic, model_ethic, timer_ethic = uncertain_model_2_ethic(C_2,V_2,C_v, S[m], B,false)
    
            # Récupération des tests croisés optimaux pour les 2 formulations
            X = value.(model[:x])
            X_ethic = value.(model_ethic[:x])
            
            # Résolution du recours pour les 2 formulations
            val_recours, model, timer = pb_recours_2(C_2,V_2,C_v, S_prime[m], false, X)
            val_ethic_recours, model_ethic, timer_ethic = pb_recours_2(C_2,V_2,C_v, S_prime[m], false, X_ethic)
        
            # Récupération des sommets testés pour les 2 formulations
            sommets = Set()
            for (i,j) in axes(X)[1]
                if abs(X[(i,j)]-1) < 1e-6
                    push!(sommets,i)
                    push!(sommets,j)
                end
            end
            sommets_ethic = Set()
            for (i,j) in axes(X_ethic)[1]
                if abs(X_ethic[(i,j)]-1) < 1e-6
                    push!(sommets_ethic,i)
                    push!(sommets_ethic,j)
                end
            end
            
            sommets = collect(sommets)
            sommets_ethic = collect(sommets_ethic)
            
            # Mise à jour de la proportion moyenne de sommets testés pour les 2 formulations
            prop += 1/M * length(setdiff(V_2,sommets))/length(V_2)*100
            prop_ethic += 1/M * length(setdiff(V_2,sommets_ethic))/length(V_2)*100
            
            # Mise à jour de la valeur objectif moyenne pour les 2 formulations
            obj_classic += 1/M * val
            obj_ethic += 1/M * val_ethic
            
            # Mise à jour de la valeur objectif moyenne du recours pour les 2 formulations
            obj_classic_recours += 1/M * val_recours
            obj_ethic_recours += 1/M * val_ethic_recours
            
            #Affichage de l'avancement du calcul (barre d'avancement)
            progress = floor(Int, (m/(M*length(B_values)) + (b-1)/length(B_values))* 20)  
            barre = "■" ^ progress                  
            vide = " " ^ (20 - progress)            
        
            # Affiche la barre et efface la ligne précédente
            print("\r[" * barre * vide * "] "*string(b)*"/"*string(length(B_values)))  
            flush(stdout) 
    
        end
    
        push!(prop_sommets,prop)
        push!(prop_sommets_ethic,prop_ethic)
        
        push!(Obj_classic,obj_classic)
        push!(Obj_ethic,obj_ethic)
    
        push!(Obj_classic_recours,obj_classic_recours)
        push!(Obj_ethic_recours,obj_ethic_recours)
    end

    println("\n")
    plot1 = plot(B_values, [prop_sommets, prop_sommets_ethic], label=["Formulation classique" "Formulation modifiée"], title="Proportion d'individus non testés", xlabel="Budget alloué", ylabel="Proportion d'individus non testés (%)")
    plot2 = plot(B_values, [Obj_classic_recours, Obj_ethic_recours], label=["Formulation classique" "Formulation modifiée"], title="Nombre de transplantation estimé", xlabel="Budget alloué", ylabel="Nombre espéré de transplantations")             
    plot!(plot1, plot2, layout=(1, 2), size=(800, 500))  

end

In [ ]:
println("Fonctions de test importées avec succès.")